In [42]:
from dotenv import load_dotenv
load_dotenv() 

True

## Load YouTube Videos


In [43]:
from langchain_community.document_loaders import YoutubeLoader

with open("videos.txt", "r") as f:
    for idx_video, video in enumerate(f):
        
        loader = YoutubeLoader.from_youtube_url(video, language="pt" )

        with open(f'transcripts/video{idx_video}.txt', "w") as f:
            pages = loader.load()
            for page in pages:
                f.write(page.page_content)
            

## Load documents from directory

In [44]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('./transcripts/', use_multithreading=True, show_progress=True)
documents = loader.load()

100%|██████████| 12/12 [00:03<00:00,  3.80it/s]


## Split Docs

In [45]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

split_docs = text_splitter.split_documents(documents)

## Create vector db

In [46]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(split_docs, OpenAIEmbeddings())

## Create the chain

In [47]:
from langchain import hub
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI


prompt = hub.pull("langchain-ai/retrieval-qa-chat")

llm = ChatOpenAI()
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever=faiss_index.as_retriever(k=4), combine_docs_chain=combine_docs_chain)
retrieval_chain.invoke({"input": "Onde estava Paulo ao escrever a carta?"})

{'input': 'Onde estava Paulo ao escrever a carta?',
 'context': [Document(page_content='escreve essa carta depois de ter recebido a visita de um amigo de um irmão epafrodito nós vamos ver isso nas próximas semanas ele provavelmente era membro da igreja de filipos e muito provavelmente um líder da igreja de filipos ele esteve em Roma por dois motivos para entregar uma oferta ao Apóstolo Paulo mas ele aproveitou obviamente deu informações para o apóstolo Paulo a respeito do dia-a-dia da igreja lá em filipos então Paulo Resolvi escrever essa carta e muito provavelmente foi o própria para fro dito que levou essa carta para filipos e ele escreve com alguns objetivos primeiro agradecer a oferta ele é grato aqueles irmãos com as ofertas daqueles irmãos Paulo estava preso numa casa alugada em Roma também para trazer algumas orientações diante de alguns assuntos teológicos alguns assuntos eclesiásticos do dia-a-dia da igreja alguns problemas que para o dito para ele por exemplo algumas lideranç